In [1]:
import pickle
import json
import torch
from glob import glob

# data loading

## BERT embedding loading

In [2]:
pt_files = sorted(glob('../bert_file/*.pt'))
pt_files_index = [int(pt.split('_')[-1].split('.')[0]) for pt in pt_files]

In [3]:
 grad_fn=<BinaryCrossEntropyBackward>),tensor(23.0769, grad_fn=<BinaryCrossEntropyBackward>)
tensor(17.6471, grad_fn=<BinaryCrossEntropyBackward>),tensor(17.6471, grad_fn=<BinaryCrossEntropyBackward>)
tensor(8.3333, grad_fn=<BinaryCrossEntropyBackward>),tensor(8.3333, grad_fn=<BinaryCrossEntropyBackward>)
tensor(15., grad_fn=<BinaryCrossEntropyBackward>),tensor(15., grad_fn=<BinaryCrossEntropyBackward>)
torch.Size([1, 24, 768]) torch.Size([1, 24, 1]) torch.Size([4, 6, 1])
tensor(23.0769, grad_fn=<BinaryCrossEntropyBackward>),tensor(23.0769, grad_fn=<BinaryCrossEntropyBackward>)
tensor(17.6471, grad_fn=<BinaryCrossEntropyBackward>),tensor(17.6471, grad_fn=<BinaryCrossEntropyBackward>)
tensor(8.3333, grad_fn=<BinaryCrossEntropyBackwlen(pt_files)

17743

In [4]:
for pt in pt_files:
    try:
        torch.load(pt)
    except:
        print(pt)

../bert_file/doc_11926.pt
../bert_file/doc_17742.pt


In [5]:
tensors = [torch.load(pt) for pt in pt_files
          if pt not in ['../bert_file/doc_11926.pt', '../bert_file/doc_17742.pt']]

## Label loading 

In [6]:
with open('../file/train.jsonl') as f:
    data = [json.loads(line) for line in f]

In [7]:
data_sample = [data[index] for index in pt_files_index]

In [8]:
# data_sample

In [9]:
label_sample = [torch.Tensor([1 if _ in data['extractive'] else 0 
                            for _ in range(len(data['article_original']))]) 
                for idx, data in enumerate(data_sample)
               if idx not in [11926, 17742]]

## train-valid split 

In [ ]:
!pip install sklearn

## Dataset 

In [88]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [89]:
class transformerDataset(Dataset):
    def __init__(self, data_x, data_y, transform_x=None, transform_y=None):
        self.data_x = data_x
        self.data_y = data_y
        
    def __len__(self):
        return len(self.data_x)
    
    def __getitem__(self, idx):
        sample_x = self.data_x[idx]
        sample_y = self.data_y[idx]
        return sample_x, sample_y

In [103]:
def pad_collate(batch):
    (xx, yy) = zip(*batch)
    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)
    yy_pad = pad_sequence(yy, batch_first=True, padding_value=0)
    return xx_pad, yy_pad, x_lens, y_len

In [104]:
# tensors = collate_fn_padd(tensors)

In [105]:
# tensors.shape

In [106]:
# label_sample

In [220]:
ds = transformerDataset(tensors, label_sample)
dl = DataLoader(ds, batch_size=4, collate_fn=pad_collate, shuffle=True, drop_last=True)

In [221]:
for d in dl:
    if d[0].shape[0] == 4:
        pass
    else:
        print(d[0].shape)

## model 

In [222]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer

In [223]:
# data_sample

In [224]:
# BERT_EMBEDDING_DIM = 768

In [225]:
# encoder_layer = TransformerEncoderLayer(d_model=BERT_EMBEDDING_DIM, nhead=2, dim_feedforward=128, dropout=0.5)
# transformer_encoder = TransformerEncoder(encoder_layer, num_layers=1)

In [226]:
# tensors[0].shape

In [227]:
# linear = nn.Linear(in_features=768, out_features=1)

In [228]:
# tensors[1].shape

In [229]:
# tensors[0].unsqueeze(0).shape

In [230]:
# trans_output = transformer_encoder(tensors[0].unsqueeze(0))

In [231]:
# trans_output.shape

In [232]:
# output = linear(trans_output)
# sigmoid = nn.Sigmoid()
# output = sigmoid(output)
# output.shape

In [233]:
# target = torch.Tensor(label_sample[0])

In [234]:
# output.shape

In [235]:
criterion = nn.BCELoss()

In [236]:
# output.shape, target.shape

In [237]:
# criterion(output, target.view(1,-1,1))

In [238]:
class PositionalEncoding(nn.Module):
    # positional encoding layer 따로 정의 
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model) # positonal encoding
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term) # 짝수는 sin 함수 적용
        pe[:, 1::2] = torch.cos(position * div_term) # 홀수는 cos 함수 적용
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [253]:
class TransformerModel(nn.Module):

    def __init__(self, ninp, nhead, nhid, n_class, dropout=0.5):
        # ntoken : number of sentence
        # ninp : the number of expected features in the input
        # nhead : the number of heads in multi-head attention model 
        # nhid : the dimension of the feedforward network model 
        # nlayers : the number of class
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, n_class)
        self.ninp = ninp
        self.lstm = nn.LSTM(ninp, ninp)
        self.linear = nn.Linear(in_features=ninp, out_features=n_class)
        self.sigmoid = nn.Sigmoid()
    
    def init_weights(self):
        initrange = 0.1
        # encoder의 nn.Embedding의 레이어 초기화
        self.encoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        src = src * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.linear(output)
        output = self.sigmoid(output)
        return output

In [259]:
criterion = nn.BCELoss()
device = 'cuda'
model = TransformerModel(ninp=768, nhead=64, nhid=4096, n_class=1).to(device)
lr = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
loss_list = []
for epoch in range(300):
    i = 0 
    for batch in dl:
        tensor = batch[0].to(device)
        label = batch[1].to(device)
        try:
            label = label.view(4, -1, 1)
            output = model(tensor)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            i += 1
            if i % 1000 == 0 :
                loss_list.append(loss)
        except Exception as e:
            print(e)
            print(tensor.shape, output.shape, label.shape)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(loss_list)